<h3> Basic summarization of the Data </h3>

In [294]:
import numpy as np
import pandas as pd
import matplotlib

In [295]:
pop_state = pd.read_csv('Population_State.csv')

In [296]:
pop_state.shape

(51, 2)

In [297]:
pop_state.head()

,region,value
0,alabama,4777326
1,alaska,711139
2,arizona,6410979
3,arkansas,2916372
4,california,37325068


In [298]:
pop_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 2 columns):
region    51 non-null object
value     51 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [299]:
dem_state = pd.read_csv('Demographics_State.csv')

In [300]:
dem_state.shape

(51, 9)

In [301]:
dem_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 9 columns):
region               51 non-null object
total_population     51 non-null int64
percent_white        51 non-null int64
percent_black        51 non-null int64
percent_asian        51 non-null int64
percent_hispanic     51 non-null int64
per_capita_income    51 non-null int64
median_rent          51 non-null int64
median_age           51 non-null float64
dtypes: float64(1), int64(7), object(1)
memory usage: 4.0+ KB


<h1>Central Limit Theorem</h1>

Given a distribution with a mean u and variance sigma^2, the sampling distribution of the mean approaches a normal distribution with a mean u and variance sigma^2/N as the sample size increases.

<h3> Estimating the population parameters of USA based on the states data </h3>

In [302]:
pop_state.describe()

,value
count,51.000000
mean,6061543.352941
std,6838088.582209
min,562803.000000
25%,1697554.500000
50%,4340167.000000
75%,6649654.500000
max,37325068.000000


Example program to illustrate central theorem

In [337]:
# TODO: Put a visualization for an example or show a demo from some other program online ?

<h2> Confidence Intervals </h2>

When one can estimate the mean of a population, it is rare to already know its standard deviation. 
Therefore, the construction of a confidence interval almost always involves the estimation of both μ and σ.

s divided by the square root of N (N is the sample size) is used as an estimate of σM

Let us calculuate the mean and standard deviation for second dataset

In [303]:
pop_states = pd.read_csv('Population_State.csv')

In [304]:
d = np.array(pop_states['value']) # todo: change 'value' column name to 'population count'

In [305]:
demograph_states = pd.read_csv('Demographics_State.csv')

In [306]:
b = np.array(demograph_states['total_population'])

In [307]:
pop_states.describe()

,value
count,51.000000
mean,6061543.352941
std,6838088.582209
min,562803.000000
25%,1697554.500000
50%,4340167.000000
75%,6649654.500000
max,37325068.000000


In [308]:
demograph_states.describe()

,total_population,percent_white,percent_black,percent_asian,percent_hispanic,per_capita_income,median_rent,median_age
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,6108560.666667,70.254902,10.823529,3.725490,10.803922,28053.803922,719.490196,37.639216
std,6904016.387730,16.116877,10.867761,5.355664,9.996038,4659.378182,189.820375,2.352367
min,570134.000000,23.000000,0.000000,1.000000,1.000000,20618.000000,448.000000,29.600000
25%,1712494.500000,59.500000,3.000000,1.000000,4.500000,24908.500000,566.000000,36.300000
50%,4361333.000000,74.000000,7.000000,2.000000,8.000000,26824.000000,664.000000,37.600000
75%,6712318.500000,82.500000,14.500000,4.000000,12.500000,30144.000000,839.000000,38.950000
max,37659181.000000,94.000000,49.000000,37.000000,47.000000,45290.000000,1220.000000,43.200000


Above comparision of the two datasets us that the sample mean estimate varies from various data sets when estimating the population parameter, in this case the population mean. 

Confidence intervals give us a way to specify a range within which the estimated population parameter could fall.

<h2> Calculating Confidence Interval for high quality weed prices for Alaska </h2>

In [309]:
# CI for high quality weed prices for Alaska
from scipy import stats
from pandas import DataFrame

weed_price = DataFrame.from_csv('Weed_Price.csv')
df = weed_price.loc['Alaska']

nv = df['HighQ']
nv

N=len(nv)
N

mean,std=(np.mean(df['HighQ']), np.std(df['HighQ']))
mean, std

ci=stats.norm.interval(0.95,loc=mean,scale=std/np.sqrt(N))
ci

(290.96643984923782, 291.99756905944804)

<h2> Calculating Confidence Interval for high quality weed prices for Florida </h2>

In [310]:
df = weed_price.loc['Florida']
nv = df['HighQ']
nv
N=len(nv)
N
mean,std=(np.mean(df['HighQ']),
                   np.std(df['HighQ']))
mean, std
ci=stats.norm.interval(0.95,loc=mean,scale=std/np.sqrt(N))
ci

(302.35844561983299, 302.78217798818491)

<h3> Bootstrapping Confidence Intervals </h3>

In [311]:
import scikits.bootstrap as bootstrap
import scipy

In [312]:
CIs = bootstrap.ci(data=b, statfunction=scipy.mean)  

In [313]:
CIs

array([ 4580812.21568627,  8599414.03921569])

In [314]:
c = np.array(pop_state['value'])

In [315]:
CIs_for_pop = bootstrap.ci(data=c, statfunction=scipy.mean)  

In [316]:
CIs_for_pop

array([ 4607019.58823529,  8534988.09803922])

<h3> Introduction to Z-score </h3>

Z-score tells us how far is each observation away from the mean of the dataset.

$$z = ({x - μ })/σ $$ 


* μ is the mean of the population;
* σ is the standard deviation of the population. 
* x is the value of each observation

In [317]:
a = np.array(pop_state['value'])

In [318]:
a

array([ 4777326,   711139,  6410979,  2916372, 37325068,  5042853,
        3572213,   900131,   605759, 18885152,  9714569,  1362730,
        1567803, 12823860,  6485530,  3047646,  2851183,  4340167,
        4529605,  1329084,  5785496,  6560595,  9897264,  5313081,
        2967620,  5982413,   990785,  1827306,  2704204,  1317474,
        8793888,  2055287, 19398125,  9544249,   676253, 11533561,
        3749005,  3836628, 12699589,  1052471,  4630351,   815871,
        6353226, 25208897,  2766233,   625498,  8014955,  6738714,
        1850481,  5687219,   562803])

* We know from summary statistics above, that the mean for this is: 6061543.352941
* Suppose, we want to know to z-score of the observation for ALABAMA, WEST VIRGINIA, CALIFORNIA 

Z-score for this can be obtained using indexes on array for each state as shown below:

In [319]:
from scipy import stats
zscore_array = stats.zscore(a)

In [320]:
zscore_array[0]

-0.18967229426808219

In [321]:
zscore_array[50]

-0.81213565284583833

In [322]:
zscore_array[4]

4.6174617039192078

<h2> A/B testing: Impact of regulation and deregulation on a couple of states </h2>

   * In A/B Testing you check two groups and see if there are any differences
   * Determine if the difference is statistically significant


In [323]:
weed_prices = pd.read_csv('Weed_Price.csv')

In [341]:
from pandas import DataFrame
df = DataFrame.from_csv('Weed_Price.csv')
sf = df.loc['Alaska'][(df.loc['Alaska']['date'] < '2014-11-02') ]
sf.describe()

# alaska data before november 2014
#https://en.wikipedia.org/wiki/Legality_of_cannabis_by_U.S._jurisdiction#.C2.A0Alaska
#State = State in US
#HighQ = Price for HighQuality Weed
#HighQN = Number of people who came on the site and pledged the value for HighQ weed on a given day.

#(Alaska prices before deregulation and after de-regulation)

,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN
count,309.000000,309.000000,309.000000,309.000000,242.000000,309.000000
mean,288.561456,302.864078,259.331036,368.831715,387.232727,29.841424
std,2.078002,25.247238,3.337891,41.994173,14.260693,3.527825
min,284.210000,252.000000,251.870000,296.000000,359.870000,26.000000
25%,287.320000,283.000000,256.510000,336.000000,375.820000,26.000000
50%,288.700000,307.000000,260.970000,371.000000,388.580000,30.000000
75%,289.810000,327.000000,261.950000,405.000000,403.330000,33.000000
max,292.100000,338.000000,263.790000,435.000000,407.310000,37.000000


In [340]:
# alaska data after november 2014
af = df.loc['Alaska'][(df.loc['Alaska']['date'] > '2014-11-02') ]
af.describe()

,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN
count,139.000000,139.000000,139.000000,139.000000,0,139.000000
mean,297.980719,361.985612,268.081871,494.604317,NaN,37.841727
std,5.459218,22.675718,3.424556,44.888044,NaN,1.580034
min,290.270000,338.000000,260.960000,437.000000,NaN,37.000000
25%,292.840000,347.000000,264.610000,461.500000,NaN,37.000000
50%,302.820000,352.000000,269.900000,481.000000,NaN,37.000000
75%,303.020000,363.500000,270.820000,502.000000,NaN,37.000000
max,303.780000,406.000000,271.500000,576.000000,NaN,41.000000


In [339]:
# Oregon data before regularization
of = df.loc['Oregon'][(df.loc['Oregon']['date'] < '2014-11-04') ]
of.describe()

,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN
count,311.000000,311.000000,311.000000,311.000000,242.000000,311.000000
mean,209.933762,1860.012862,185.487781,1600.221865,170.173388,76.209003
std,1.689441,135.259249,2.385111,155.090563,5.117401,7.706180
min,207.660000,1606.000000,181.500000,1329.000000,162.910000,61.000000
25%,208.640000,1754.000000,183.525000,1472.000000,165.750000,70.000000
50%,209.550000,1853.000000,185.330000,1605.000000,169.420000,77.000000
75%,211.335000,1988.000000,188.460000,1751.500000,175.200000,82.000000
max,213.540000,2076.000000,189.150000,1857.000000,178.050000,86.000000


In [342]:
# Oregon data after regularization
of = df.loc['Oregon'][(df.loc['Oregon']['date'] > '2014-11-04') ]
of.describe()

,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN
count,137.000000,137.000000,137.000000,137.000000,0,137.000000
mean,205.238905,2250.401460,180.280073,2098.992701,NaN,98.598540
std,1.727567,148.092998,1.209925,172.594582,NaN,8.583037
min,202.020000,2083.000000,178.040000,1871.000000,NaN,87.000000
25%,205.190000,2146.000000,180.270000,1973.000000,NaN,91.000000
50%,205.570000,2204.000000,180.860000,2049.000000,NaN,98.000000
75%,206.580000,2253.000000,181.090000,2125.000000,NaN,99.000000
max,207.470000,2533.000000,181.730000,2406.000000,NaN,113.000000


In [328]:
dcf = df.loc['District of Columbia'][(df.loc['District of Columbia']['date'] < '2014-11-04') ]
dcf.describe()

,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN
count,311.000000,311.000000,311.000000,311.000000,242.000000,311.000000
mean,348.308360,529.276527,290.894920,442.154341,210.563554,45.041801
std,2.173169,53.350521,2.333702,55.026052,1.951919,3.081923
min,339.150000,431.000000,284.260000,343.000000,205.810000,39.000000
25%,346.370000,483.500000,288.850000,395.000000,209.830000,41.000000
50%,348.640000,526.000000,291.000000,442.000000,209.830000,47.000000
75%,350.220000,576.500000,292.520000,495.000000,211.380000,47.000000
max,352.280000,614.000000,296.060000,533.000000,216.650000,48.000000


In [329]:
dacf = df.loc['District of Columbia'][(df.loc['District of Columbia']['date'] > '2014-11-04') ]
dacf.describe()

,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN
count,137.000000,137.000000,137.000000,137.000000,0,137.000000
mean,347.872774,678.781022,282.278175,613.540146,NaN,50.072993
std,1.481656,46.465312,0.928332,64.200400,NaN,2.147622
min,345.010000,620.000000,280.500000,539.000000,NaN,48.000000
25%,347.690000,645.000000,281.830000,567.000000,NaN,49.000000
50%,348.470000,663.000000,282.100000,589.000000,NaN,49.000000
75%,348.860000,687.000000,282.550000,626.000000,NaN,50.000000
max,349.850000,764.000000,286.990000,729.000000,NaN,54.000000


In [330]:
calicf = df.loc['California'][(df.loc['California']['date'] > '2014-11-04') ]
calicf.describe()

,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN
count,137.000000,137.000000,137.000000,137.000000,0,137.000000
mean,243.456715,16901.036496,189.319051,19704.693431,NaN,1120.197080
std,0.998957,892.047804,0.828273,1350.024665,NaN,65.682373
min,241.840000,15752.000000,187.850000,17871.000000,NaN,1029.000000
25%,242.930000,16231.000000,188.870000,18662.000000,NaN,1069.000000
50%,243.680000,16646.000000,189.210000,19378.000000,NaN,1106.000000
75%,244.150000,17092.000000,190.110000,20015.000000,NaN,1138.000000
max,244.940000,18492.000000,190.830000,22027.000000,NaN,1232.000000


In [331]:
calicf = df.loc['California'][(df.loc['California']['date'] < '2014-11-04') ]
calicf.describe()

,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN
count,311.000000,311.000000,311.000000,311.000000,242.000000,311.000000
mean,246.222765,14083.790997,192.129100,15473.517685,190.795992,912.739550
std,1.234917,1085.216014,0.795382,1505.169027,1.586186,76.095396
min,244.940000,12021.000000,190.820000,12724.000000,187.830000,770.000000
25%,245.190000,13189.000000,191.530000,14162.500000,189.420000,845.500000
50%,245.710000,14157.000000,192.010000,15561.000000,191.075000,929.000000
75%,246.980000,15143.000000,192.900000,16923.500000,192.200000,989.000000
max,248.820000,15720.000000,193.630000,17815.000000,193.880000,1026.000000


<h5> Insights from the above analysis: </h5>
    
   * Price went up in Alaska after regularization. 
   * But reduced in other districts.
   * People buying increased in all districts.

<h2> Hypothesis Testing </h2>

In [332]:
sf = DataFrame.from_csv('Weed_Price.csv')
#sf = wp[wp['LowQ'] > 0]

# function to calculate the sum of weed data
def sumOfHighQualityWeedPrice(array):
    sum = 0.0
    for i in array:
        sum = sum + i
    return sum

# Alaska before november
alaska_data_below_november = sf[sf['date'] < '2014-11-02']
alaska_data = np.array(alaska_data_below_november.loc['Alaska']['HighQ'])
alaska_data_size = len(alaska_data)
alaska_high_quality_weed_price_mean_before_november = sumOfHighQualityWeedPrice(alaska_data)/alaska_data_size

# Alaska after november
alaska_data_after_november = sf[sf['date'] > '2014-11-02']
alaska_after_nov_data = np.array(alaska_data_after_november.loc['Alaska']['HighQ'])
alaska_after_nov_data_size = len(alaska_after_nov_data)
alaska_high_quality_weed_price_mean_after_november = sumOfHighQualityWeedPrice(alaska_after_nov_data)/alaska_after_nov_data_size

oregon_data_below_november = sf[sf['date'] < '2014-11-04']
oregon_data = np.array(oregon_data_below_november.loc['Oregon']['HighQ'])
oregon_data_size = len(oregon_data)
oregon_high_quality_weed_price_mean_before_november = sumOfHighQualityWeedPrice(oregon_data)/oregon_data_size

oregon_data_after_november = sf[sf['date'] > '2014-11-04']
oregon_after_data = np.array(oregon_data_after_november.loc['Oregon']['HighQ'])
oregon_after_data_size = len(oregon_after_data)
oregon_high_quality_weed_price_mean_after_november = sumOfHighQualityWeedPrice(oregon_after_data)/oregon_after_data_size

# District of Columbia
dc_data_below_november = sf[sf['date'] < '2014-11-04']
dc_data = np.array(dc_data_below_november.loc['District of Columbia']['HighQ'])
dc_data_size = len(dc_data)
dc_high_quality_weed_price_mean_before_november = sumOfHighQualityWeedPrice(dc_data)/dc_data_size

dc_data_after_november = sf[sf['date'] > '2014-11-04']
dc_after_data = np.array(dc_data_after_november.loc['District of Columbia']['HighQ'])
dc_after_data_size = len(dc_after_data)
dc_high_quality_weed_price_mean_after_november = sumOfHighQualityWeedPrice(dc_after_data)/dc_after_data_size

# means for before november data for alaska, oregon, District of Columbia
# make a table for these data
# TODO: Check if the data passes requirements for z-test
# increase in alaska weed price after regularization (1% increase).

# Since we want to check if the price of the weed went down, this is one-tailed hypothesis test
# The z-test
(oregon_high_quality_weed_price_mean_after_november - oregon_high_quality_weed_price_mean_before_november)/1.727567
oregon_stats = 1 - stats.norm.cdf(-2.7176120801038213)
oregon_stats #p-value > 0.1: There is little or no evidence against H0

# DC Z-TEST
(dc_high_quality_weed_price_mean_after_november - dc_high_quality_weed_price_mean_before_november)/1.481656
dc_stats = 1 - stats.norm.cdf(1.0)
dc_stats #p-value > 0.1: There is little or no evidence against H0

# Alaska Z-TEST
(alaska_high_quality_weed_price_mean_after_november - alaska_high_quality_weed_price_mean_before_november)/5.459218
alaska_stats = 1 - stats.norm.cdf(1.7253868802786354) # 0.01 < p-value < 0.05: There is strong evidence against H0
alaska_stats, dc_stats, oregon_stats

# For the first two null hypothesis holds true. 
# For Alaska, null hypothesis is rejected as the p-value is less than 0.05

#So, find the value where alaska will agree with the null hypothesis

# GET STANDARD DEVIATIONS
#oregon_data_after_november.loc['Oregon']['HighQ'].describe()
#dc_data_after_november.loc['District of Columbia']['HighQ'].describe()
#alaska_data_after_november.loc['Alaska']['HighQ'].describe()



(0.042228887044626462, 0.15865525393145707, 0.99671225596586954)

<h2> T-tests for hypothesis testing: </h2>

In [333]:
# t-test for indepdence btw two categories in dataset

from scipy.stats import ttest_ind

cat1 = alaska_data_below_november.loc['Alaska']
cat2 = alaska_data_after_november.loc['Alaska']
#(-26.401039029301675, 3.4667367833940785e-93)

cat1 = oregon_data_below_november.loc['Oregon']
cat2 = oregon_data_after_november.loc['Oregon']
#(26.914061651283518, 1.7639735830507177e-95)

cat1 = dc_data_below_november.loc['District of Columbia']
cat2 = dc_data_after_november.loc['District of Columbia']
ttest_ind(cat1['HighQ'], cat2['HighQ'])
#(2.1368196148483887, 0.03315614856932287)

# US AVERAGE WEED PRICE = 286.35$

(2.1368196148483887, 0.03315614856932287)

<h2> T-Test to test whether means of two populations are significantly different from each other (unpaired data)</h2>

Let us test for "Is there a difference in High Quality Weed prices after regularization ?"

In [334]:
# Mean for alaska before regularization
x1 = alaska_high_quality_weed_price_mean_before_november

# Mean for alaska after regularization
x2 = alaska_high_quality_weed_price_mean_after_november

# Let us compare the variance of each population
# Let s1 be the variance for alaska before regularization
variance1 = np.var(alaska_data)

# Let s2 be the variance for alaska after regualarization
variance2 = np.var(alaska_after_nov_data)

# Comparing tells us that the variance is un-equal
variance1,variance2

(4.3041192383825368, 29.58864552559384)

* T-statistic for comparing two means of paired data (independent) is given by:

$$t = (x1-x2) / \sqrt{(s1^2/n1) + (s2^2/n2)}$$

In [343]:
n1 = alaska_data_size
n2 = alaska_after_nov_data_size

def calculateTStatistics(mean1,mean2,var1,var2,samplesize1,samplesize2):
    t = (mean1-mean2)/np.sqrt(((var1)/samplesize1) + ((var2)/samplesize2))
    return t

df1 = alaska_data_size-1
df2 = alaska_after_nov_data_size- 1

t = calculateTStatistics(x1, x2, variance1, variance2, n1, n2)
print "FOR Alaska:"
print "----------"
print "degrees of freedom for alaska is:", df1 - df2
print "t-statistic for alaska is:", t
print "The P-Value is < 0.00001. The result is significant at p < 0.05"
print "The P-Value is < 0.00001. The result is significant at p < 0.01"
print " "


# Let us do this for DC data

xdf1 = dc_data_size-1
xdf2 = dc_after_data_size- 1
dcn1 = dc_data_size
dcn2 = dc_after_data_size

# Mean for alaska before regularization
dcx1 = dc_high_quality_weed_price_mean_before_november

# Mean for alaska after regularization
dcx2 = dc_high_quality_weed_price_mean_after_november

# Let us compare the variance of each population
# Let s1 be the variance for alaska before regularization
dvariance1 = np.var(dc_data)

# Let s2 be the variance for alaska after regualarization
dvariance2 = np.var(dc_after_data)

dc_t = calculateTStatistics(dcx1, dcx2, dvariance1, dvariance2, dcn1, dcn2)
print " "
print "FOR DC"
print "------"
print "degrees of freedom for DC is:", xdf1 - xdf2
print "t-statistic for DC is:", dc_t
print "The P-Value is 0.013525. The result is significant at p < 0.05"
print "The P-Value is 0.013525. The result is not significant at p < 0.01."
print " "
print " "
# Oregon data

odf1 = oregon_data_size-1
odf2 = oregon_after_data_size- 1
on1 = oregon_data_size
on2 = oregon_after_data_size

# Mean for alaska before regularization
ox1 = oregon_high_quality_weed_price_mean_before_november

# Mean for alaska after regularization
ox2 = oregon_high_quality_weed_price_mean_after_november

# Let us compare the variance of each population
# Let s1 be the variance for alaska before regularization
ovariance1 = np.var(oregon_data)

# Let s2 be the variance for alaska after regualarization
ovariance2 = np.var(oregon_after_data)

o_t = calculateTStatistics(ox1, ox2, ovariance1, ovariance2, on1, on2)

print "FOR Oregon:"
print "-----------"
print "degrees of freedom for Oregon is:", odf1 - odf2
print "t-statistic for Oregon is:", o_t
print " "

print "The P-Value is < 0.00001. The result is significant at p < 0.05"
print "The P-Value is < 0.00001. The result is significant at p < 0.01"
print " "
print " "
print "------"
print "The Null hypothesis H0 that there is no difference btw means of populations is rejected at level 1%"
print "But, there is some price change at significance level of 5% when price is regularized for DC data at 5%."
print "Here, we reject the null hypothesis H0 in favor of alternative hypothesis that there is statistical difference HA"

FOR Alaska:
----------
degrees of freedom for alaska is: 170
t-statistic for alaska is: -19.7787180634
The P-Value is < 0.00001. The result is significant at p < 0.05
The P-Value is < 0.00001. The result is significant at p < 0.01
 
 
FOR DC
------
degrees of freedom for DC is: 174
t-statistic for DC is: 2.47222088427
The P-Value is 0.013525. The result is significant at p < 0.05
The P-Value is 0.013525. The result is not significant at p < 0.01.
 
 
FOR Oregon:
-----------
degrees of freedom for Oregon is: 174
t-statistic for Oregon is: 26.762901661
 
The P-Value is < 0.00001. The result is significant at p < 0.05
The P-Value is < 0.00001. The result is significant at p < 0.01
 
 
------
The Null hypothesis H0 that there is no difference btw means of populations is rejected at level 1%
But, there is some price change at significance level of 5% when price is regularized for DC data at 5%.
Here, we reject the null hypothesis H0 in favor of alternative hypothesis that there is statistic

<h2> Something to think about: Which of these give smaller p-values ? </h2>
   
   * Smaller effect size
   * Smaller standard error
   * Smaller sample size
   * Higher variance
   
   Answer: A smaller standard-error

<h2> Chi-Square Testing </h2>

* Chi-Square testing is used to conduct hypothesis testing on categorial variables data.

In year 2014, so many number of ppl bought low quality, mid quality and highly quality weed. Some NGO took action to educate ppl to buy high quality weed. 

In 2015, we see the numbers (Sum up total number of ppl bought for each of those three categories for both the years). And figure out if response is valid or not.

* TODO: What is a chi-square distribution.

<h2> Chi-Square test for goodness fit </h2>

$$ X^2 = \sum (O - E)^2/E $$

* O is observed frequency
* E is expected frequency
* X is the chi-square statistic

In [400]:
sf = DataFrame.from_csv('Weed_Price.csv')
# number of people who brought high quality weed in Alaska in 2014
alaska_data_in_2014 = sf[sf['date'] < '2015-01-01']
alaska_data_filtered_2014 = alaska_data_in_2014[alaska_data_in_2014['date'] > '2013-12-31']
alaska_14_numbers = np.array(alaska_data_filtered_2014.loc['Alaska']['HighQN'])
alaska_data_filtered_2014.loc['Alaska']['HighQN'].describe()


count    364.000000
mean     310.491758
std       27.235782
min      252.000000
25%      286.000000
50%      316.500000
75%      331.000000
max      350.000000
Name: HighQN, dtype: float64

In [401]:
# number of people who brought low quality weed in Alaska in 2014
alaska_low_14_numbers = np.array(alaska_data_filtered_2014.loc['Alaska']['LowQN'])
alaska_data_filtered_2014.loc['Alaska']['LowQN'].describe()


count    364.000000
mean      31.074176
std        4.159690
min       26.000000
25%       26.000000
50%       32.000000
75%       34.000000
max       37.000000
Name: LowQN, dtype: float64

In [386]:
# number of people who brought high quality weed in Alaska in 2015
alaska_data_in_2015 = sf[sf['date'] > '2014-12-31']
alaska_high_15_numbers = np.array(alaska_data_in_2015.loc['Alaska']['HighQN'])
alaska_data_in_2015.loc['Alaska']['HighQN'].describe()


count     80.000000
mean     374.500000
std       22.629095
min      350.000000
25%      355.000000
50%      363.000000
75%      398.750000
max      406.000000
Name: HighQN, dtype: float64

In [402]:
# number of people who brought low quality weed in Alaska in 2015
alaska_data_in_2015 = sf[sf['date'] > '2014-12-31']
alaska_low_15_numbers = np.array(alaska_data_in_2015.loc['Alaska']['LowQN'])
alaska_data_in_2015.loc['Alaska']['LowQN'].describe()


count    80.000000
mean     38.462500
std       1.855159
min      37.000000
25%      37.000000
50%      37.000000
75%      41.000000
max      41.000000
Name: LowQN, dtype: float64

In [399]:
O = alaska_high_15_numbers

len(O)
np.mean(O)

l = [0] * len(O)
e = np.array(l)
e = e + 374.5

chi_test_2015 = scipy.stats.chisquare(O,e)

O = alaska_14_numbers
l = [0] * len(O)
e = np.array(l)
e = e + 374.5

chi_test_2014 = scipy.stats.chisquare(O,e)

chi_test_2015, chi_test_2014

((108.02136181575435, 0.01674509201460693), (4701.1935914552741, 0.0))

In [406]:
O = alaska_low_15_numbers

len(O)
np.mean(O)

l = [0] * len(O)
e = np.array(l)
e = e + 38.46

chi_test_low_2015 = scipy.stats.chisquare(O,e)

O = alaska_low_14_numbers
l = [0] * len(O)
e = np.array(l)
e = e + 38.46

chi_test_low_2014 = scipy.stats.chisquare(O,e)

chi_test_low_2015, chi_test_low_2014



((7.0693707748309933, 1.0), (679.598086323453, 1.6115618694114632e-21))

Like this we can find out at which value of price, there was significant effect for each State in 2014 and 2015
for different qualities of weed.